# Chapter 10. Data Aggregation and Group Operations

## 10.1 GroupBy Mechanics

Hadley Wickham, an author of many popular packages for the R programming language, coined the term split-apply-combine for describing group operations. 
![Split Apply Combine](images/split-apply-combine.png)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                  'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'data1' : [1, 2, 3, 4, 5],
                  'data2' : [10, 20, 30, 40, 50]})

df

,key1,key2,data1,data2
0,a,one,1,10
1,a,two,2,20
2,b,one,3,30
3,b,two,4,40
4,a,one,5,50


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
# the name of the index is 'key1'
grouped.mean()

key1
a    2.666667
b    3.500000
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     3
      two     2
b     one     3
      two     4
Name: data1, dtype: int64

In [6]:
means.unstack()

key2,one,two
key1,,
a,3,2
b,3,4


In [7]:
# You can use arrays from outside to group data
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    2.0
            2006    3.0
Ohio        2005    2.5
            2006    5.0
Name: data1, dtype: float64

In [8]:
# non numerical data 'key2' is hidden (nuisance column)
df.groupby('key1').mean()

,data1,data2
key1,,
a,2.666667,26.666667
b,3.500000,35.000000


In [9]:
df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    one       3     30
     two       2     20
b    one       3     30
     two       4     40

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2  data1  data2
0    a  one      1     10
1    a  two      2     20
4    a  one      5     50
b
  key1 key2  data1  data2
2    b  one      3     30
3    b  two      4     40


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2  data1  data2
0    a  one      1     10
4    a  one      5     50
('a', 'two')
  key1 key2  data1  data2
1    a  two      2     20
('b', 'one')
  key1 key2  data1  data2
2    b  one      3     30
('b', 'two')
  key1 key2  data1  data2
3    b  two      4     40


In [13]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2  data1  data2
 0    a  one      1     10
 1    a  two      2     20
 4    a  one      5     50,
 'b':   key1 key2  data1  data2
 2    b  one      3     30
 3    b  two      4     40}

### Selecting a Column or Subset of Columns

```
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
The code above is syntactic sugar for:
```
df['data1'].groupby(df['key1'])
df['data2'].groupby(df['key2'])
```

### Grouping with Dicts and Series

In [14]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

people

,a,b,c,d,e
Joe,-2.129733,-1.089311,1.254966,0.297443,-1.461928
Steve,-1.746190,-1.115945,0.057529,1.275346,-0.579826
Wes,-0.636334,NaN,NaN,-0.887541,0.046808
Jim,-1.498528,-2.088729,-0.533896,-1.136251,-0.337698
Travis,-0.919138,0.669749,0.613213,0.402092,-0.566478


In [15]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [17]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,1.552409,-4.680972
Steve,1.332875,-3.441961
Wes,-0.887541,-0.589526
Jim,-1.670147,-3.924955
Travis,1.015306,-0.815868


In [18]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [19]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3
